## Unification

Unifiication determines if assertions match rules, and if so, what variable subsitutions are needed to get them to match. This is a key concept to understand and powerful once you get the hang of it. Let's go through a few examples.

For these examples, we only need the unification functions and not the full engine.

In [1]:
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))
import thoughts.unification
import pprint

## Exact Text Match

The most basic example of unification is when a text string directly matches another text string. Note that the result is an empty dict, meaning that it found no conflicts, but also found no subsitutions required in order to unify these two terms.

In [2]:
unification = thoughts.unification.unify_strings("hello", "hello")
pprint.pprint(unification)

{}


## Text Does Not Match

If the strings are unable to unify (do not match), then the function will return a None, indicating a NULL intersection of these two terms.

In [3]:
unification = thoughts.unification.unify_strings("hello", "hi")
pprint.pprint(unification)

None


## Text with Variables

Variables can appear within strings. The function will return a unification variable for text that is covered by the variable.

In [4]:
unification = thoughts.unification.unify_strings("my name is ?name", "my name is jeremy")
pprint.pprint(unification)

{'?name': 'jeremy'}


## Text with Multiple Variables

This matching is not limited to just one variable. You can include multiple wildcard markers that begin with "?" and the function will return a unification variable for every variable it is able to match in the text, based on the surrounding words.

This can be a very powerful technique for simple parsing, where you know the overall structure of the phrase ahead of time.

In [5]:
pattern = "?premise therefore ?conclusion"
candidate = "i think therefore i am"
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{'?premise': 'i think', '?conclusion': 'i am'}


## Fact Matching

Where things get interesting is unifying (matching) facts with other facts. The unification function will return a unification if every attribute in the pattern fact exists in the candidate fact, and if each attribute from the candidate fact unifies with the corresponding attribute from the pattern fact.

In the example below, both the pattern and candidate facts have a "sky" and "temperature" attribute, and the values of this attributes match between both facts.

In [6]:
pattern = {"sky": "cloudy", "temperature": "cold"}

candidate = {"sky": "cloudy", "temperature": "cold"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{}


## When Facts Do No Match

When the unification function is not able to unify the facts, it will return None.

In the example below, the candidate's temperature is "warm" and the pattern's temperature is "cold", so these facts do not unify.

In [7]:
candidate = {"sky": "cloudy", "temperature": "warm"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

None


## Facts Can Use Variables Too

Variables can appear in facts to allow for wildcard matching. The unfication algorithm will unify these according to the rules described above for unifying strings.

In [8]:
pattern = {"sky": "?sky", "temperature": "cold"}

candidate = {"sky": "cloudy", "temperature": "cold"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{'?sky': 'cloudy'}


## Variables Can Appear in Pattern or Candidate Terms

Variables can appear in either the pattern term or the candidate term.

In [9]:
pattern = {"sky": "cloudy", "temperature": "cold"}

candidate = {"sky": "?sky", "temperature": "cold"}
unification = thoughts.unification.unify(pattern, candidate)
pprint.pprint(unification, sort_dicts=False)

{'?sky': 'cloudy'}


## Variables Can Appear in Both Pattern and Candidate Terms

Variables can also appear in both the pattern and candidate terms at the same time. The unification function will return a variable binding as a merged dictionary between both terms.

In [10]:
pattern = {"sky": "?sky", "temperature": "cold"}

candidate = {"sky": "cloudy", "temperature": "?temp"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{'?sky': 'cloudy', '?temp': 'cold'}


## Variables can Unify with Other Variables

Variables can unify with other variables. When this occurs, the true value of the variable must be provided outside of the unification algorithm or evaluated for truth later.

In [11]:
pattern = {"sky": "?sky1", "temperature": "?temp1"}

candidate = {"sky": "?sky2", "temperature": "?temp2"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{'?sky2': '?sky1', '?temp2': '?temp1'}


## Facts Can Use Multi-Variable Strings

Facts can contain string terms which have more than one variable, same as the unification rules above. These can be useful for parsing where you want to limit a pattern to only match when some other semantic condition occurs.

In [12]:
pattern = {"phrase": "?person ate ?food", "location": "?location"}

candidate = {"phrase": "jeremy ate the sandwich", "location": "restaurant"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{'?person': 'jeremy', '?food': 'the sandwich', '?location': 'restaurant'}


## Candidate Terms Must Have All Attributes from the Pattern Term

A candidate term must have all attributes that are present in the pattern term.

In the example below, the pattern term contains a "wind" attribute, which is not present in the candidate term.

This is a key idea - that patterns will require that the incoming candidate term have ALL the attributes which are present. This allows you to create rules with similar patterns but with some slight discriminating difference.

In [13]:
pattern = {"sky": "?sky1", "temperature": "?temp1", "wind": "SSW"}

candidate = {"sky": "?sky2", "temperature": "?temp2"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

None


## Candidate Terms Can Have More Terms than the Pattern

On the flip side, candidate terms can contain more terms than pattern terms. This is useful where you want to pass additional information along with the candidate term throughout the assertion chain. The pattern will simply ignore this additional information but will pass it along so that other rules can use the information if needed.

In [14]:
pattern = {"sky": "?sky1", "temperature": "?temp1"}

candidate = {"sky": "?sky2", "temperature": "?temp2", "wind": "SSW"}
unification = thoughts.unification.unify(candidate, pattern)
pprint.pprint(unification, sort_dicts=False)

{'?sky2': '?sky1', '?temp2': '?temp1'}


In [16]:
rules = [
    {"item": "brown", "sem": {"color": "BROWN"}},
    {"item": "dog", "sem": {"entity": "DOG"}},

    {"when":    {"input": "test"},
     "then":    [{"#lookup": "$brown.sem", "#into": "?sem-art"},
                {"#lookup": "$dog.sem", "#into": "?sem-noun"},
                {"cat": "np", "sem": {"#combine": ["?sem-noun", "?sem-art"]} }]
    }
]